In [1]:
import numpy as np
import networkx as nx
import matplotlib.pylab as plt
import pandas as pd



%matplotlib inline


#assuming the importation of data as pandas data frames before here

#====begin making graph/network====


dfP=pd.read_csv('./data/final.csv')
dfP.rename(columns={'Category':'crimeCount'}, inplace=True)

## set true to draw all of the plots
drawPlots = False;




In [2]:
#Making networks
from networkx.algorithms import bipartite
teamsGraph=nx.Graph()
teams=dfP.team.unique()


for i in teams:
	teamsGraph.add_node(i,team=1,bipartite=0,crimes=-1) #team name

names=dfP.name.unique()	



for name in names:#number of players
  
    nameTemp=dfP[(dfP.name==name)]

    playerCrimes=nameTemp.crimeCount.unique()
  
    crimee=0
    for crim in playerCrimes:
        if crim==1:
            crimee=1
    teamsGraph.add_node(name,crimes=crimee, bipartite=1)
    
    playerTeams=nameTemp.team.unique()
    for team in playerTeams:
        teamsGraph.add_edge(team,name)

		
    


In [3]:
#Actual network analysis


interestNodes=[n for n,m in teamsGraph.nodes(data=True) if (m['bipartite']==0 or m['crimes']==1)]
crimeNodes=[n for n,m in teamsGraph.nodes(data=True) if (m['crimes']==1)]
notCrimeNodes=[n for n,m in teamsGraph.nodes(data=True) if (m['crimes']==0)]

teamNodes=[n for n,m in teamsGraph.nodes(data=True) if (m['crimes']==-1)]
listOfEdges = [edge for edge in teamsGraph.edges()]

## see https://stackoverflow.com/questions/16910538/efficient-extraction-of-a-subgraph-according-to-some-edge-attribute-in-networkx
crimeNet=teamsGraph.subgraph(interestNodes)
crimeEdges= [edge for edge in crimeNet.edges()]
#
#
##cool network analysis 
C=nx.clustering(crimeNet)
B=nx.betweenness_centrality(crimeNet)
crAdjMat=crimeNetAdjMat = nx.adjacency_matrix(crimeNet)
#
#ax(B[ x)
import operator
maxBet=max(B.items(), key=operator.itemgetter(1))[0]
minBet=min(B.items(), key=operator.itemgetter(1))[0]





In [4]:
if drawPlots:
    pos = nx.spring_layout(crimeNet)
    nx.draw_networkx_nodes(crimeNet, pos, nodelist=crimeNodes, cmap=plt.get_cmap('jet'), node_color = 'r',node_size=10)
    nx.draw_networkx_nodes(crimeNet, pos, nodelist=teamNodes, cmap=plt.get_cmap('jet'),node_color = 'g',node_size=10)

    nx.draw_networkx_edges(crimeNet, pos, edgelist=crimeEdges, arrows=False)

In [5]:
if drawPlots:  
    pos = nx.spring_layout(teamsGraph)
    nx.draw_networkx_nodes(teamsGraph, pos, nodelist=notCrimeNodes, cmap=plt.get_cmap('jet'), node_color = 'b',node_size=10)
    nx.draw_networkx_nodes(teamsGraph, pos, nodelist=crimeNodes, cmap=plt.get_cmap('jet'), node_color = 'r',node_size=10)
    nx.draw_networkx_nodes(teamsGraph, pos, nodelist=teamNodes, cmap=plt.get_cmap('jet'),node_color = 'g',node_size=10)
    #nx.draw_networkx_labels(teamsGraph, pos)
    nx.draw_networkx_edges(teamsGraph, pos, edgelist=listOfEdges, arrows=False)
    #plt.figure(figsize=)
    #plt.savefig("fullNet.png", dpi=1000)
    #plt.savefig("fullNet.pdf")
    plt.show()


In [6]:
playerNodes = (notCrimeNodes + crimeNodes)

uniTeamsGraph = bipartite.projected_graph(teamsGraph, playerNodes)

uniEdges = uniTeamsGraph.edges()


In [7]:
if drawPlots:
    pos = nx.spring_layout(uniTeamsGraph)
    nx.draw_networkx_nodes(uniTeamsGraph, pos, nodelist=notCrimeNodes, cmap=plt.get_cmap('jet'), node_color = 'b',node_size=10)
    nx.draw_networkx_nodes(uniTeamsGraph, pos, nodelist=crimeNodes, cmap=plt.get_cmap('jet'), node_color = 'r',node_size=10)
    nx.draw_networkx_edges(uniTeamsGraph, pos, edgelist=uniEdges, arrows=False)


In [8]:
uniCrimesGraph = bipartite.projected_graph(crimeNet, crimeNodes)
uniCrimeEdges = uniCrimesGraph.edges()

if drawPlots:
    nx.draw(uniCrimesGraph)


In [9]:
crimeGraph=nx.Graph()
crimeList = ['Alcohol','Animal Abuse','Assault / Battery', 'DUI','Disorderly conduct','Domestic Violence','Drugs','Gun','License / Traffic','Murder / Manslaughter','Resisting','Sex','Other','Theft / Burglary']

for i in crimeList:
    crimeGraph.add_node(i,issaCrime=1)
    
dfC = dfP[(dfP.crimeCount >= 1)]

for i in dfC.name.unique():
    crimeGraph.add_node(i, issaCrime=0)


def createCrimeNodes(i, row):
    for crime in crimeList:
        if row[crime] >= 1:
            crimeGraph.add_edge(crime, row['name'])
[createCrimeNodes(i, row) for i, row in dfC.iterrows()]

# print(dict([n for n, m in crimeGraph.nodes(data=True) if (m['issaCrime']==1)]))
labels = dict((key, key) for (key, value) in crimeGraph.nodes(data=True) if (value['issaCrime']==1))

crimeNodes =[n for n,m in crimeGraph.nodes(data=True) if (m['issaCrime']==1)]
playNodes =[n for n,m in crimeGraph.nodes(data=True) if (m['issaCrime']==0)]
listOfCrimeEdges= [edge for edge in crimeGraph.edges()]


In [10]:
#printing graph that links players crimes to the crimes
if drawPlots:
    pos = nx.spring_layout(crimeGraph)
    nx.draw_networkx_nodes(crimeGraph, pos, nodelist=crimeNodes, cmap=plt.get_cmap('jet'), node_color = 'r',node_size=20)
    nx.draw_networkx_nodes(crimeGraph, pos, nodelist=playNodes, cmap=plt.get_cmap('jet'),node_color = 'g',node_size=10)
    nx.draw_networkx_edges(crimeGraph, pos, edgelist=listOfCrimeEdges, arrows=False)
    nx.draw_networkx_labels(crimeGraph, pos, labels=labels)



In [14]:
###Degree centrality by team in crime net
D = crimeNet.degree()
D = dict(D)
degrees = sorted(set(D.values()))
degree_agg = [list(D.values()).count(x) for x in degrees]
    
teamDegrees = {}
for key in list((D))[:32]:
    teamDegrees[key] = D[key]

##identify team nodes for centrality measures
teamNodes = set(n for n,d in crimeNet.nodes(data=True) if d['bipartite']==0)

##Closeness by team in crime net
TC = bipartite.closeness_centrality(crimeNet, teamNodes)

teamCloseness = {}
for key in list((TC))[:32]:
    teamCloseness[key] = TC[key]

##Betweenness by team in crime net
TB = bipartite.betweenness_centrality(crimeNet, teamNodes)

teamBetweenness = {}
for key in list((TB))[:32]:

    teamBetweenness[key] = TB[key]


if drawPlots:
    plt.figure(figsize=(10,8))
    plt.plot(degrees, degree_agg)

    plt.legend('')
    plt.ylabel('Number of nodes (Player/Team)')
    plt.xlabel('Degree')
    plt.ylim(1,200)

    plt.title("Degree metrics for Crime Network")
    plt.savefig("DegreeMet.png")

if drawPlots:
    plt.figure(figsize=(10,8))
    plt.bar(range(len(teamBetweenness)), teamBetweenness.values())
    plt.xticks(range(len(teamBetweenness)), teamBetweenness.keys(), rotation=90)
    plt.title("Betweenness Centrality for NFL teams in the Crime Network")
    plt.ylabel('Betweenness')
    plt.xlabel("Team")
    plt.savefig("BetweenMet.png")
    plt.show()


{'NYJ': 33, 'LA': 27, 'SF': 36, 'CAR': 22, 'CLE': 47, 'ARI': 30, 'NE': 43, 'BUF': 28, 'MIA': 50, 'PIT': 26, 'BAL': 41, 'CIN': 40, 'TEN': 41, 'JAC': 37, 'HOU': 21, 'IND': 35, 'KC': 31, 'OAK': 40, 'LAC': 31, 'DEN': 49, 'PHI': 26, 'DAL': 38, 'WAS': 44, 'NYG': 34, 'MIN': 38, 'GB': 36, 'DET': 37, 'CHI': 41, 'NO': 44, 'ATL': 23, 'TB': 50, 'SEA': 38, 'David Boston': 4, 'Dennis McKinley': 1, 'Michael Pittman': 3, 'Jake Plummer': 2, 'Yusuf Scott': 1, 'Wendell Bryant': 1, 'Neil Rackers': 3, 'Jeremy Bridges': 2, 'Leon Joe': 3, 'David Macklin': 4, 'Peppi Zellner': 3, 'J.J. Arrington': 1, 'Alan Branch': 4, 'Bryan Robinson': 4, 'A.J. Jefferson': 2, 'Joey Porter': 3, 'Daryl Washington': 1, 'Quentin Groves': 5, 'Bobby Massie': 2, 'John Abraham': 3, 'Bryan McCann': 5, 'Alex Okafor': 1, 'Frostee Rucker': 3, 'Kevin Faulk': 1, 'Terry Glenn': 3, 'Bobby Hamilton': 4, 'Adrian Klemm': 2, 'Ty Law': 4, 'Lawyer Milloy': 4, 'Kenyatta Jones': 2, 'Ben Kelly': 2, 'Mike Vrabel': 3, 'Daniel Graham': 4, 'Rabih Abdullah

In [37]:

playerEigen = nx.eigenvector_centrality_numpy(uniTeamsGraph)
sorted_EU = sorted(EU.items(), key=operator.itemgetter(1))

len(sorted_EU)
sorted_EU = dict(sorted_EU)
print(sorted_EU)

{'Johnny Maxey': 0.0030903337516452515, 'John Malecki': 0.0030903337516452519, 'Bud Dupree': 0.0030903337516452519, 'Javon Hargrave': 0.0030903337516452519, 'Joe Burnett': 0.0030903337516452523, 'Cameron Heyward': 0.0030903337516452523, 'Derek Moye': 0.0030903337516452523, 'Hines Ward': 0.0030903337516452523, 'Courtney Hawkins': 0.0030903337516452528, 'Willie Parker': 0.0030903337516452528, 'Dallas Baker': 0.0030903337516452528, 'David Paulson': 0.0030903337516452528, 'Demarcus Ayers': 0.0030903337516452528, 'Myron Bell': 0.0030903337516452532, 'Mathias Nkwenti': 0.0030903337516452532, 'Brett Keisel': 0.0030903337516452532, 'Stevenson Sylvester': 0.0030903337516452532, 'John Clay': 0.0030903337516452532, "Da'Mon Cromartie-Smith": 0.0030903337516452532, 'Sammie Coates': 0.0030903337516452532, 'Doran Grant': 0.0030903337516452532, 'Ryan Shazier': 0.0030903337516452532, 'Leterrius Walton': 0.0030903337516452532, "Le'Veon Bell": 0.0030903337516452532, 'Lee Flowers': 0.0030903337516452536, 

In [38]:
playerDegrees = uniTeamsGraph.degree
playerDegrees= dict(playerDegrees)
#playerBetweenness = nx.betweenness_centrality(uniTeamsGraph) #takes too long


In [39]:
# not really sure the difference between uniTeamsGraph and teamsGraph but Brady was using uni
#ETG=nx.eigenvector_centrality_numpy(teamsGraph)
#sorted_ETG = sorted(ETG.items(), key=operator.itemgetter(1))
#print(sorted_ETG)

In [43]:
##Metrics to add to data


##takes too long for the wholegraph
##closeness in one-mode graph
#cl = nx.closeness_centrality(uniTeamsGraph)
#bt = nx.betweenness_centrality(uniTeamsGraph)

def matchPlayerNetworkMeasures(name):
    dfP.loc[(dfP.name == name), 'player_Degrees'] = playerDegrees.get(name)
    dfP.loc[(dfP.name == name), 'player_Eigen'] = playerEigen.get(name)
    #dfP.loc[(dfP.name == name), 'player_Betweenness'] = playerBetweenness[team] #takes too long

[matchPlayerNetworkMeasures(name) for name in dfP.name.unique()]


def matchNetworkMeasures(team):
    dfP.loc[(dfP.team == team), 'Team_Degrees'] = teamDegrees[team]
    dfP.loc[(dfP.team == team), 'Team_Closeness'] = teamCloseness[team]
    dfP.loc[(dfP.team == team), 'Team_Betweenness'] = teamBetweenness[team]

[matchNetworkMeasures(team) for team in dfP.team.unique()]

#matchNetworkMeasures('NYJ')

#dfP.loc[(dfP.team == "NYJ"), ['Team_Degrees','Team_Closeness','Team_Betweenness']]

    
dfP.to_csv('./data/final_networked_by_team.csv', index=False)

PermissionError: [Errno 13] Permission denied: './data/final_networked_by_team.csv'

In [41]:
dfP.head()

,No,name,Age,Pos,G,GS,Wt,Ht,College,BirthDate,...,Disorderly conduct,Sex,Theft / Burglary,Murder / Manslaughter,Animal Abuse,Team_Degrees,Team_Closeness,Team_Betweenness,player_Degrees,player_Eigen
0,45.0,Josh Baker,25.0,NaN,11.0,0.0,244.0,75.0,"Delware, Northwest Missouri State",725365.0,...,NaN,NaN,NaN,NaN,NaN,33.0,0.369705,0.050261,452.0,0.006007
1,34.0,Chris Smith,24.0,NaN,3.0,0.0,186.0,70.0,"Highland J.C., Northern Illionois",725545.0,...,NaN,NaN,NaN,NaN,NaN,27.0,0.362264,0.037319,979.0,0.013362
2,26.0,Tramaine Brock,23.0,NaN,11.0,0.0,197.0,70.0,"Mississippi Gulf Coast J.C.,Bellhaven",725969.0,...,NaN,NaN,NaN,NaN,NaN,36.0,0.367816,0.050009,419.0,0.005294
3,26.0,Tramaine Brock,24.0,NaN,16.0,0.0,197.0,70.0,"Mississippi Gulf Coast J.C.,Bellhaven",725969.0,...,NaN,NaN,NaN,NaN,NaN,36.0,0.367816,0.050009,419.0,0.005294
4,26.0,Tramaine Brock,25.0,RCB,16.0,7.0,197.0,70.0,"Mississippi Gulf Coast J.C.,Bellhaven",725969.0,...,NaN,NaN,NaN,NaN,NaN,36.0,0.367816,0.050009,419.0,0.005294


In [42]:
#null checking
null_columns=dfP.columns[dfP.isnull().any()]
dfP[null_columns].isnull().sum()

No                         578
Age                          6
Pos                      11362
GS                          32
Wt                           7
Ht                       32467
College                     18
BirthDate                    6
AV                           1
Draft_Team                9506
Draft_Round               9506
Draft_Pick                9506
Draft_Year                9529
crimeCount               32603
DUI                      32603
Other                    32603
Drugs                    32603
Domestic Violence        32603
Resisting                32603
Assault / Battery        32603
Gun                      32603
Alcohol                  32603
License / Traffic        32603
Disorderly conduct       32603
Sex                      32603
Theft / Burglary         32603
Murder / Manslaughter    32603
Animal Abuse             32603
dtype: int64